In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

## WAQI Dataset

In [ ]:
waqi_2017H1 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2017H1.csv', header=4)
waqi_2018H1 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2018H1.csv', header=4)
waqi_2019Q1 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2019Q1.csv', header=4)
waqi_2019Q2 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2019Q2.csv', header=4)
waqi_2019Q3 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2019Q3.csv', header=4)
waqi_2019Q4 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2019Q4.csv', header=4)
waqi_2020Q1 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2020Q1.csv', header=4)
waqi_2020Q2 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2020Q2.csv', header=4)
waqi_2020Q3 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2020Q3.csv', header=4)
waqi_2020Q4 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2020Q4.csv', header=4)
waqi_2021 = pd.read_csv('/work/city_raw_data/waqi/waqi-covid-2020.csv', header=4) # in fact it's 2021

waqi_combined = pd.concat([waqi_2017H1, waqi_2018H1, waqi_2019Q1, waqi_2019Q2, waqi_2019Q3, waqi_2019Q4, waqi_2020Q1, waqi_2020Q2, waqi_2020Q3, waqi_2020Q4, waqi_2021])

In [ ]:
def clean(city_name, output_name, country_code=None):
    # Extract data for selected city
    if country_code is not None:
        city_original = waqi_combined[(waqi_combined['City']==city_name)&(waqi_combined['Country']==country_code)]
    else : 
        city_original = waqi_combined[waqi_combined['City']==city_name]
    city_original['Date'] = pd.to_datetime(city_original['Date'])
    city_original.index = city_original['Date']
    city_original.index.name = 'date'
    city_original = city_original.sort_index()
    city_original = city_original.drop_duplicates(keep='last')
    city_original = city_original[(city_original.index>='2017-01-01')&(city_original.index<='2021-03-31')]

    # Organize the pollutents into columns
    city_pm25_df = city_original[city_original['Specie']=='pm25']
    city_pm25_df = city_pm25_df[['median']].rename(columns={'median':'pm25'})
    city_pm10_df = city_original[city_original['Specie']=='pm10']
    city_pm10_df = city_pm10_df[['median']].rename(columns={'median':'pm10'})
    city_co_df = city_original[city_original['Specie']=='co']
    city_co_df = city_co_df[['median']].rename(columns={'median':'co'})
    city_no2_df = city_original[city_original['Specie']=='no2']
    city_no2_df = city_no2_df[['median']].rename(columns={'median':'no2'})
    city_so2_df = city_original[city_original['Specie']=='so2']
    city_so2_df = city_so2_df[['median']].rename(columns={'median':'so2'})
    city_o3_df = city_original[city_original['Specie']=='o3']
    city_o3_df = city_o3_df[['median']].rename(columns={'median':'o3'})

    city_df = pd.concat([city_pm25_df, city_pm10_df, city_o3_df, city_no2_df, city_so2_df, city_co_df], axis=1)

    # forward fill null values
    city_df = city_df.fillna(method='ffill')

    # Calculate AQI
    cityAQI = city_df.assign(AQI=city_df.max(axis=1))

    # Output
    writer = cityAQI
    writer.to_csv(f'city_cleaned_data/{output_name}_AQI.csv',index=True)

    return cityAQI

### Beijing

In [ ]:
beijing_data = clean('Beijing', 'beijing')
beijing_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,469.0,388.0,2.5,67.0,4.1,59.5,469.0
2017-01-02,291.0,197.0,7.0,52.5,6.1,36.2,291.0
2017-01-03,382.0,293.0,2.9,74.5,8.2,58.7,382.0
2017-01-04,422.0,407.0,2.9,76.1,7.6,75.8,422.0
2017-01-05,260.0,165.0,1.7,54.7,7.1,48.8,260.0
...,...,...,...,...,...,...,...
2021-03-20,72.0,73.0,26.9,5.5,1.6,5.5,73.0
2021-03-21,30.0,26.0,30.5,4.2,1.1,1.9,30.5
2021-03-22,87.0,127.0,17.9,14.2,1.1,2.8,127.0


### Hong Kong

In [ ]:
hong_kong_data = clean('Hong Kong', 'hong_kong')
hong_kong_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,73.0,33.0,16.7,27.4,1.9,6.1,73.0
2017-01-02,80.0,39.0,15.7,33.4,4.8,7.1,80.0
2017-01-03,94.0,44.0,16.2,33.2,4.2,7.5,94.0
2017-01-04,77.0,35.0,25.2,21.1,3.7,6.0,77.0
2017-01-05,77.0,33.0,9.2,35.6,3.8,8.1,77.0
...,...,...,...,...,...,...,...
2021-03-20,57.0,21.0,10.0,22.7,2.4,5.2,57.0
2021-03-21,70.0,39.0,11.6,17.6,2.4,7.0,70.0
2021-03-22,63.0,33.0,22.5,15.8,2.1,7.2,63.0


### Johannesburg

In [ ]:
johannesburg_data = clean('Johannesburg', 'johannesburg')
johannesburg_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2018-12-31,63.0,22.0,6.0,4.2,1.1,4.0,63.0
2019-01-01,57.0,20.0,6.1,4.2,1.6,4.1,57.0
2019-01-02,61.0,23.0,10.6,1.9,0.6,4.4,61.0
2019-01-03,34.0,15.0,8.2,2.3,1.6,3.7,34.0
2019-01-04,50.0,19.0,9.7,2.8,1.1,3.9,50.0
...,...,...,...,...,...,...,...
2021-03-19,79.0,35.0,4.2,3.7,1.6,10.5,79.0
2021-03-20,75.0,35.0,10.1,3.6,1.5,4.2,75.0
2021-03-21,61.0,24.0,9.7,3.4,1.4,13.4,61.0


### London

In [ ]:
london_data = clean('London', 'london', 'GB')
london_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,44.0,15.0,16.1,13.3,2.3,2.8,44.0
2017-01-02,33.0,14.0,11.2,18.6,1.6,2.2,33.0
2017-01-03,51.0,18.0,3.3,28.5,3.0,3.6,51.0
2017-01-04,35.0,15.0,7.0,26.2,2.6,2.2,35.0
2017-01-05,66.0,30.0,2.2,37.8,3.7,3.8,66.0
...,...,...,...,...,...,...,...
2021-03-19,25.0,10.0,23.4,8.6,1.3,8.2,25.0
2021-03-20,42.0,14.0,15.3,13.9,1.3,7.9,42.0
2021-03-21,34.0,12.0,20.7,6.5,1.8,7.0,34.0


### Milan

In [ ]:
milan_data = clean('Milan', 'milan')
milan_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,188.0,99.0,9.8,38.0,3.1,0.1,188.0
2017-01-02,153.0,58.0,2.5,42.5,3.1,0.1,153.0
2017-01-03,153.0,59.0,16.3,45.7,3.1,0.1,153.0
2017-01-04,122.0,51.0,24.8,40.7,3.1,0.1,122.0
2017-01-05,38.0,12.0,27.3,19.2,3.1,0.1,38.0
...,...,...,...,...,...,...,...
2021-03-17,13.0,11.0,32.5,41.6,3.1,0.1,41.6
2021-03-18,38.0,18.0,31.3,31.1,3.1,0.1,38.0
2021-03-19,57.0,23.0,31.7,27.5,2.6,0.1,57.0


### Rome

In [ ]:
rome_data = clean('Rome', 'rome')
rome_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,151.0,59.0,4.5,21.1,0.8,0.1,151.0
2017-01-02,104.0,43.0,2.5,33.0,0.4,0.1,104.0
2017-01-03,46.0,13.0,9.8,17.9,0.4,0.1,46.0
2017-01-04,59.0,22.0,7.4,26.6,0.1,0.1,59.0
2017-01-05,46.0,13.0,15.1,13.3,0.3,0.1,46.0
...,...,...,...,...,...,...,...
2021-03-18,46.0,14.0,22.4,11.0,0.3,0.1,46.0
2021-03-19,46.0,17.0,20.8,13.3,0.2,0.1,46.0
2021-03-20,30.0,11.0,25.6,8.3,0.3,0.1,30.0


### São Paulo

In [ ]:
sao_paulo_data = clean('São Paulo', 'sao_paulo')
sao_paulo_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,46.0,19.0,16.3,8.3,1.1,4.6,46.0
2017-01-02,38.0,16.0,14.7,12.4,0.6,4.6,38.0
2017-01-03,57.0,20.0,17.5,14.7,1.1,4.6,57.0
2017-01-04,50.0,19.0,16.7,19.7,1.1,4.6,50.0
2017-01-05,50.0,20.0,18.3,15.6,0.6,4.6,50.0
...,...,...,...,...,...,...,...
2021-03-19,55.0,18.0,15.5,13.3,0.6,3.7,55.0
2021-03-20,57.0,20.0,18.3,7.8,1.1,3.7,57.0
2021-03-21,57.0,20.0,19.9,7.8,1.1,3.7,57.0


### Seoul

In [ ]:
seoul_data = clean('Seoul', 'seoul')
seoul_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,152.0,64.0,3.2,42.6,7.2,10.0,152.0
2017-01-02,159.0,72.0,12.8,38.9,8.6,10.0,159.0
2017-01-03,137.0,64.0,8.8,38.9,8.6,8.9,137.0
2017-01-04,97.0,52.0,4.8,49.1,8.6,10.0,97.0
2017-01-05,82.0,44.0,7.3,38.0,7.2,7.8,82.0
...,...,...,...,...,...,...,...
2021-03-20,46.0,29.0,23.3,20.4,4.3,4.5,46.0
2021-03-21,97.0,44.0,31.2,11.2,4.3,5.6,97.0
2021-03-22,57.0,33.0,25.6,15.8,4.3,4.5,57.0


### Shanghai

In [ ]:
shanghai_data = clean('Shanghai', 'shanghai')
shanghai_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,98.0,45.0,10.6,34.8,8.7,7.3,98.0
2017-01-02,155.0,63.0,9.0,32.5,10.7,10.9,155.0
2017-01-03,168.0,69.0,24.8,33.0,9.0,10.9,168.0
2017-01-04,144.0,54.0,18.7,33.4,10.2,8.2,144.0
2017-01-05,80.0,27.0,30.5,20.9,7.1,7.3,80.0
...,...,...,...,...,...,...,...
2021-03-20,85.0,30.0,15.1,17.4,2.1,5.5,85.0
2021-03-21,156.0,67.0,31.3,13.3,3.1,6.4,156.0
2021-03-22,82.0,52.0,24.4,19.7,3.6,5.2,82.0


### Sydney

In [ ]:
sydney_data = clean('Sydney', 'sydney')
sydney_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,43.0,23.0,19.2,3.8,1.5,1.2,43.0
2017-01-02,30.0,17.0,16.8,1.9,1.5,2.3,30.0
2017-01-03,28.0,17.0,12.1,2.8,1.5,2.3,28.0
2017-01-04,17.0,11.0,11.2,2.8,1.5,2.3,17.0
2017-01-05,13.0,11.0,10.4,4.7,2.9,1.2,13.0
...,...,...,...,...,...,...,...
2021-03-20,13.0,16.0,14.5,2.8,1.5,2.3,16.0
2021-03-21,19.0,19.0,14.5,2.8,1.5,1.2,19.0
2021-03-22,11.0,8.0,11.2,6.5,1.5,2.3,11.2


### Wuhan

In [ ]:
wuhan_data = clean('Wuhan', 'wuhan')
wuhan_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-02,182.0,100.0,13.0,30.7,7.6,12.6,182.0
2017-01-03,198.0,117.0,8.2,41.2,8.7,15.0,198.0
2017-01-04,223.0,137.0,1.7,40.7,6.6,19.0,223.0
2017-01-05,80.0,26.0,4.1,18.8,2.6,15.7,80.0
2017-01-06,61.0,17.0,10.2,15.1,2.1,6.9,61.0
...,...,...,...,...,...,...,...
2021-03-20,119.0,55.0,22.0,11.0,3.6,6.9,119.0
2021-03-21,124.0,95.0,24.8,11.0,4.1,5.9,124.0
2021-03-22,80.0,70.0,24.4,16.5,4.1,4.2,80.0


## EPA Dataset
Data for the three selected cities has specificity. Cannot use a single function to clean them.

### Greater New York City

In [ ]:
# Read in Greater NYC data from epa/
gnyc_co_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_co-csv.csv')
gnyc_no2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_no2-csv.csv')
gnyc_o3_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_o3-csv.csv')
gnyc_pm10_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_pm10-csv.csv')
gnyc_pm25_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_pm25-csv.csv')
gnyc_so2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_so2-csv.csv')
gnyc_aqi_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gnyc_aqi-csv.csv')

In [ ]:
# Convert the 'Date' column to datatype: datetime
gnyc_co_original['Date'] = pd.to_datetime(gnyc_co_original['Date'])
gnyc_co_original.index = gnyc_co_original['Date']
gnyc_no2_original['Date'] = pd.to_datetime(gnyc_no2_original['Date'])
gnyc_no2_original.index = gnyc_no2_original['Date']
gnyc_pm10_original['Date'] = pd.to_datetime(gnyc_pm10_original['Date'])
gnyc_pm10_original.index = gnyc_pm10_original['Date']
gnyc_pm25_original['Date'] = pd.to_datetime(gnyc_pm25_original['Date'])
gnyc_pm25_original.index = gnyc_pm25_original['Date']
gnyc_so2_original['Date'] = pd.to_datetime(gnyc_so2_original['Date'])
gnyc_so2_original.index = gnyc_so2_original['Date']
gnyc_o3_original['Date'] = pd.to_datetime(gnyc_o3_original['Date'])
gnyc_o3_original.index = gnyc_o3_original['Date']
gnyc_aqi_original['Date'] = pd.to_datetime(gnyc_aqi_original['Date'])
gnyc_aqi_original.index = gnyc_aqi_original['Date']

In [ ]:
# Extract the pollutants, combine them into one dataframe
gnyc_co_df = gnyc_co_original['Carbon Monoxide AQI Value']
gnyc_no2_df = gnyc_no2_original['Nitrogen Dioxide AQI Value']
gnyc_pm10_df = gnyc_pm10_original['PM10 AQI Value']
gnyc_pm25_df = gnyc_pm25_original['PM2.5 AQI Value']
gnyc_o3_df = gnyc_o3_original['Ozone AQI Value']
gnyc_so2_df = gnyc_so2_original['Sulfur Dioxide AQI Value']
gnyc_aqi_df = gnyc_aqi_original['AQI Value']
gnyc_combined_df = pd.concat([gnyc_pm25_df, gnyc_pm10_df,gnyc_o3_df,gnyc_no2_df,gnyc_so2_df,gnyc_co_df,gnyc_aqi_df], axis = 1)
gnyc_combined_df.index.name = 'date'
gnyc_combined_df_sorted = gnyc_combined_df.sort_index()
gnyc_combined_df_sorted = gnyc_combined_df_sorted.rename(columns={'PM2.5 AQI Value': 'pm25','PM10 AQI Value':'pm10','Ozone AQI Value':'o3','Nitrogen Dioxide AQI Value':'no2','Sulfur Dioxide AQI Value':'so2','Carbon Monoxide AQI Value':'co', 'AQI Value':'AQI'})
gnyc_combined_df_sorted

,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2011-01-01,119,NaN,31,53.0,47.0,22.0,119
2011-01-02,76,NaN,29,48.0,41.0,20.0,76
2011-01-03,57,25.0,35,42.0,52.0,7.0,57
2011-01-04,86,NaN,25,49.0,37.0,9.0,86
2011-01-05,72,NaN,32,48.0,46.0,9.0,72
...,...,...,...,...,...,...,...
2021-03-18,40,NaN,36,NaN,NaN,NaN,40
2021-03-19,27,NaN,40,NaN,NaN,NaN,40
2021-03-20,37,NaN,42,NaN,NaN,NaN,42


In [ ]:
# Since Greater NYC pm10 is taken twice in a week. We forward fill the empty value with the existing measurment data for the week.
gnyc_combined_df_sorted['pm10'] = gnyc_combined_df_sorted['pm10'].fillna(method='ffill')

In [ ]:
# Check the null value in the data
pd.isnull(gnyc_combined_df_sorted).sum()
# pm10 data has 2 null values, for 2011. It's okay.
# Others are null for Feb, March 2021. Hopefully will be filled by updated dataset.


pm25     0
pm10     2
o3       0
no2     50
so2     50
co      49
AQI      0
dtype: int64

In [ ]:
# Select the period for output
gnyc_final = gnyc_combined_df_sorted.loc['2017-01-01':'2021-01-31']
# Check if there is any null value
pd.isnull(gnyc_final).sum()

pm25    0
pm10    0
o3      0
no2     0
so2     0
co      0
AQI     0
dtype: int64

In [ ]:
# Convert the pollutant columns into int
for i in range (1,len(gnyc_final.columns)):
    gnyc_final[gnyc_final.columns[i]]=gnyc_final[gnyc_final.columns[i]].astype(int)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Output
gnyc_final.to_csv("city_cleaned_data/gnyc_AQI.csv",index=True)
gnyc_final.head()

,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,59,11,34,44,4,11,59
2017-01-02,60,11,33,43,17,13,60
2017-01-03,43,11,32,42,1,7,43
2017-01-04,46,7,32,27,3,6,46
2017-01-05,49,7,32,28,3,6,49


### Greater Los Angeles

In [ ]:
# Read in Greater LA data from epa/
gla_co_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_co-csv.csv')
gla_no2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_no2-csv.csv')
gla_o3_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_o3-csv.csv')
gla_pm10_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_pm10-csv.csv')
gla_pm25_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_pm25-csv.csv')
gla_so2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_so2-csv.csv')
gla_aqi_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gla_aqi-csv.csv')

In [ ]:
# Convert the 'Date' column to datatype: datetime
gla_co_original['Date'] = pd.to_datetime(gla_co_original['Date'])
gla_co_original.index = gla_co_original['Date']
gla_no2_original['Date'] = pd.to_datetime(gla_no2_original['Date'])
gla_no2_original.index = gla_no2_original['Date']
gla_pm10_original['Date'] = pd.to_datetime(gla_pm10_original['Date'])
gla_pm10_original.index = gla_pm10_original['Date']
gla_pm25_original['Date'] = pd.to_datetime(gla_pm25_original['Date'])
gla_pm25_original.index = gla_pm25_original['Date']
gla_so2_original['Date'] = pd.to_datetime(gla_so2_original['Date'])
gla_so2_original.index = gla_so2_original['Date']
gla_o3_original['Date'] = pd.to_datetime(gla_o3_original['Date'])
gla_o3_original.index = gla_o3_original['Date']
gla_aqi_original['Date'] = pd.to_datetime(gla_aqi_original['Date'])
gla_aqi_original.index = gla_aqi_original['Date']
gla_aqi_original

,Date,AQI Value,Main Pollutant,Site Name,Site ID,Source
Date,,,,,,
2011-01-01,2011-01-01,99,PM2.5,Burbank,06-037-1002,AQS
2011-01-02,2011-01-02,64,PM2.5,Burbank,06-037-1002,AQS
2011-01-03,2011-01-03,44,PM2.5,Burbank,06-037-1002,AQS
2011-01-04,2011-01-04,52,NO2,Long Beach (hudson),06-037-4006,AQS
2011-01-05,2011-01-05,58,PM2.5,Burbank,06-037-1002,AQS
...,...,...,...,...,...,...
2021-03-18,2021-03-18,64,Ozone,Lancaster-division Street,06-037-9033,AirNow
2021-03-19,2021-03-19,51,PM2.5,North Hollywood (noho),06-037-4010,AirNow
2021-03-20,2021-03-20,45,PM2.5,North Hollywood (noho),06-037-4010,AirNow


In [ ]:
# Extract the pollutants, combine them into one dataframe
gla_co_df = gla_co_original['Carbon Monoxide AQI Value']
gla_no2_df = gla_no2_original['Nitrogen Dioxide AQI Value']
gla_pm10_df = gla_pm10_original['PM10 AQI Value']
gla_pm25_df = gla_pm25_original['PM2.5 AQI Value']
gla_o3_df = gla_o3_original['Ozone AQI Value']
gla_so2_df = gla_so2_original['Sulfur Dioxide AQI Value']
gla_aqi_df = gla_aqi_original['AQI Value']
gla_combined_df = pd.concat([gla_pm25_df, gla_pm10_df,gla_o3_df,gla_no2_df,gla_so2_df,gla_co_df,gla_aqi_df], axis = 1)
gla_combined_df.index.name = 'date'
gla_combined_df_sorted = gla_combined_df.sort_index()
gla_combined_df_sorted = gla_combined_df_sorted.rename(columns={'PM2.5 AQI Value': 'pm25','PM10 AQI Value':'pm10','Ozone AQI Value':'o3','Nitrogen Dioxide AQI Value':'no2','Sulfur Dioxide AQI Value':'so2','Carbon Monoxide AQI Value':'co', 'AQI Value':'AQI'})
gla_combined_df_sorted

,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2011-01-01,99,35.0,35,40.0,7.0,7.0,99
2011-01-02,64,17.0,33,41.0,1.0,15.0,64
2011-01-03,44,15.0,36,37.0,0.0,9.0,44
2011-01-04,51,20.0,36,52.0,1.0,15.0,52
2011-01-05,58,23.0,38,51.0,9.0,18.0,58
...,...,...,...,...,...,...,...
2021-03-18,56,NaN,64,NaN,NaN,NaN,64
2021-03-19,51,NaN,43,NaN,NaN,NaN,51
2021-03-20,45,NaN,45,NaN,NaN,NaN,45


In [ ]:
# Select the period for output
gla_final = gla_combined_df_sorted.loc['2017-01-01':'2020-12-31']
# Check if there is null value
pd.isnull(gla_final).sum()


pm25    0
pm10    0
o3      0
no2     0
so2     0
co      0
AQI     0
dtype: int64

In [ ]:
# Output
gla_final.to_csv("city_cleaned_data/gla_AQI.csv",index=True)
gla_final.head()


,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,145,28.0,39,30.0,1.0,27.0,145
2017-01-02,59,10.0,42,36.0,1.0,13.0,59
2017-01-03,66,17.0,32,55.0,3.0,17.0,66
2017-01-04,72,20.0,32,48.0,4.0,20.0,72
2017-01-05,43,10.0,31,36.0,1.0,13.0,43


### Greater Miami

In [ ]:
# Read in Greater Miami data from epa/
gmia_co_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_co.csv')
gmia_no2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_no2.csv')
gmia_o3_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_o3.csv')
gmia_pm10_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_pm10.csv')
gmia_pm25_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_pm25.csv')
gmia_so2_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_so2.csv')
gmia_aqi_original = pd.read_csv('/work/city_raw_data/epa/2011_2021_gmia_aqi.csv')

In [ ]:
# Convert the 'Date' column to datatype: datetime
gmia_co_original['Date'] = pd.to_datetime(gmia_co_original['Date'])
gmia_co_original.index = gmia_co_original['Date']
gmia_no2_original['Date'] = pd.to_datetime(gmia_no2_original['Date'])
gmia_no2_original.index = gmia_no2_original['Date']
gmia_pm10_original['Date'] = pd.to_datetime(gmia_pm10_original['Date'])
gmia_pm10_original.index = gmia_pm10_original['Date']
gmia_pm25_original['Date'] = pd.to_datetime(gmia_pm25_original['Date'])
gmia_pm25_original.index = gmia_pm25_original['Date']
gmia_so2_original['Date'] = pd.to_datetime(gmia_so2_original['Date'])
gmia_so2_original.index = gmia_so2_original['Date']
gmia_o3_original['Date'] = pd.to_datetime(gmia_o3_original['Date'])
gmia_o3_original.index = gmia_o3_original['Date']
gmia_aqi_original['Date'] = pd.to_datetime(gmia_aqi_original['Date'])
gmia_aqi_original.index = gmia_aqi_original['Date']
gmia_aqi_original

,Date,AQI Value,Main Pollutant,Site Name,Site ID,Source
Date,,,,,,
2011-01-01,2011-01-01,64,PM2.5,Royal Palm Beach,12-099-0009,AQS
2011-01-02,2011-01-02,68,PM2.5,Belle Glade,12-099-0008,AQS
2011-01-03,2011-01-03,50,PM2.5,Miami Fire Station,12-086-1016,AQS
2011-01-04,2011-01-04,45,PM2.5,Belle Glade,12-099-0008,AQS
2011-01-05,2011-01-05,51,PM2.5,Belle Glade,12-099-0008,AQS
...,...,...,...,...,...,...
2021-04-14,2021-04-14,51,Ozone,Rosenstiel,12-086-0027,AirNow
2021-04-15,2021-04-15,51,Ozone,Vista View Park,12-011-0033,AirNow
2021-04-16,2021-04-16,59,PM2.5,KENDALL,12-086-0034,AirNow


In [ ]:
# Extract the pollutants, combine them into one dataframe
gmia_co_df = gmia_co_original['Carbon Monoxide AQI Value']
gmia_no2_df = gmia_no2_original['Nitrogen Dioxide AQI Value']
gmia_pm10_df = gmia_pm10_original['PM10 AQI Value']
gmia_pm25_df = gmia_pm25_original['PM2.5 AQI Value']
gmia_o3_df = gmia_o3_original['Ozone AQI Value']
gmia_so2_df = gmia_so2_original['Sulfur Dioxide AQI Value']
gmia_aqi_df = gmia_aqi_original.iloc[:,1]
gmia_combined_df = pd.concat([gmia_pm25_df, gmia_pm10_df,gmia_o3_df,gmia_no2_df,gmia_so2_df,gmia_co_df,gmia_aqi_df], axis = 1)
gmia_combined_df.index.name = 'date'
gmia_combined_df_sorted = gmia_combined_df.sort_index()
gmia_combined_df_sorted = gmia_combined_df_sorted.rename(columns={'PM2.5 AQI Value': 'pm25','PM10 AQI Value':'pm10','Ozone AQI Value':'o3','Nitrogen Dioxide AQI Value':'no2','Sulfur Dioxide AQI Value':'so2','Carbon Monoxide AQI Value':'co'})
gmia_combined_df_sorted.columns.values[6] = 'AQI'
gmia_combined_df_sorted

,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2011-01-01,64.0,19.0,41.0,25.0,4.0,5.0,64
2011-01-02,68.0,22.0,42.0,31.0,20.0,7.0,68
2011-01-03,50.0,27.0,34.0,31.0,6.0,8.0,50
2011-01-04,45.0,21.0,35.0,29.0,7.0,10.0,45
2011-01-05,51.0,19.0,34.0,35.0,6.0,11.0,51
...,...,...,...,...,...,...,...
2021-04-14,44.0,NaN,51.0,NaN,NaN,NaN,51
2021-04-15,40.0,NaN,51.0,NaN,NaN,NaN,51
2021-04-16,59.0,NaN,50.0,NaN,NaN,NaN,59


In [ ]:
# Select the period for output
gmia_final = gmia_combined_df_sorted.loc['2017-01-01':'2020-12-31']
# Use forward fill for the missing data
gmia_final = gmia_final.fillna(method='ffill')
# Check if there is null value
pd.isnull(gmia_final).sum()


pm25    0
pm10    0
o3      0
no2     0
so2     0
co      0
AQI     0
dtype: int64

In [ ]:
# Output
gmia_final.to_csv("city_cleaned_data/gmia_AQI.csv",index=True)
gmia_final.head()

,pm25,pm10,o3,no2,so2,co,AQI
date,,,,,,,
2017-01-01,76.0,20.0,43.0,6.0,0.0,7.0,76
2017-01-02,49.0,21.0,33.0,24.0,0.0,7.0,49
2017-01-03,54.0,13.0,28.0,25.0,0.0,7.0,54
2017-01-04,40.0,7.0,29.0,23.0,1.0,6.0,40
2017-01-05,52.0,14.0,50.0,38.0,1.0,11.0,52


## Oxford Policy Data

In [ ]:
policy_original = pd.read_csv('/work/city_raw_data/policy/OxCGRT_latest-csv.csv')

In [ ]:
def get_lockdown_dates(city, country, region=None):
    if region is None:
        city_original = policy_original[policy_original['CountryName']==country]
    else:
        city_original = policy_original[(policy_original['CountryName']==country)&(policy_original['RegionName']==region)]
    city_lockdown = city_original[(city_original['C6_Stay at home requirements']==2)]
    start_row = city_lockdown.iloc[0]
    start_date = start_row['Date']
    # print(start_date)
    end_row = city_lockdown.iloc[-1]
    end_date = end_row['Date']
    # print(end_date)
    days = (dt.datetime.strptime(str(end_date), "%Y%m%d") - dt.datetime.strptime(str(start_date), "%Y%m%d")).days + 1
    # print(city_lockdown.shape[0], days)
    if days == city_lockdown.shape[0]:
        return (start_date, end_date)
    else:
        return 'Multiple lockdowns or no lockdown, need to check'


# test = get_lockdown_dates(city='gnyc', country='United States', region='New York')
# test

In [ ]:
# Oxford policy data has sub-national data for United States and United Kingdom, here we try to extrat lockdown date from them.
lockdown_dates = dict()
lockdown_dates['gnyc'] = get_lockdown_dates(city='gnyc', country='United States', region='New York')
lockdown_dates['gla'] = get_lockdown_dates(city='gla', country='United States', region='California')
lockdown_dates['gmia'] = get_lockdown_dates(city='gmia', country='United States', region='Florida')
lockdown_dates['london'] = get_lockdown_dates(city='london', country='United Kingdom', region='England')

lockdown_dates

{'gnyc': (20200322, 20200607),
 'gla': 'Multiple lockdowns or no lockdown, need to check',
 'gmia': 'Multiple lockdowns or no lockdown, need to check',
 'london': 'Multiple lockdowns or no lockdown, need to check'}

### Curated Lockdown Dates

#### Greater Miami
start date: **03/24/2020** (according to https://www.clickorlando.com/news/local/2020/03/20/timeline-the-spread-of-coronavirus-in-florida/, supported by Oxford data. 

end date: **05/18/2020** ("full phase one reopening", according to Oxford data, supported by https://www.clickorlando.com/news/local/2020/03/20/timeline-the-spread-of-coronavirus-in-florida/

Statewide stay-at-home order is issued from **04/03/2020** - **04/30/2020** according to government doc" https://www.flgov.com/wp-content/uploads/orders/2020/EO_20-91-compressed.pdf) 


#### Greater LA
first time start date: **03/20/2020** (according to https://www.nbclosangeles.com/news/coronavirus/california-coronavirus-pandemic-timeline-key-events/2334100/, https://www.nbclosangeles.com/news/local/los-angeles-safer-at-home-rules/2332257/ ), 

first time end date: **05/29/2020** (according to https://abc7news.com/timeline-of-coronavirus-us-covid-19-bay-area-sf/6047519/, https://www.nbclosangeles.com/news/coronavirus/california-coronavirus-pandemic-timeline-key-events/2334100/, ), 

second time start date: **11/27/2020** (according to https://www.nbclosangeles.com/news/coronavirus/california-coronavirus-pandemic-timeline-key-events/2334100/, Oxford says 11/30/2020)

second time end date: **01/25/2021** (according to https://www.nbclosangeles.com/news/coronavirus/california-coronavirus-pandemic-timeline-key-events/2334100/, https://covid19.ca.gov/stay-home-except-for-essential-needs/#current-restrictions, supported by Oxford data)

### Greater NYC
start date: **03/22/2020** 

end date: **06/07/2020** 
dates are supported as extracted by the code from Oxford data {'gnyc': (20200322, 20200607),

### London
start date: 1st **03/23/2020**, 2nd **11/05/2020**, 3rd **01/06/2021**

end date: 1st **05/10/2020**, 2nd **12/02/2020**, 3rd **03/06/2021** ('stay local' order replaced the previous 'stay at home' order)

(https://www.instituteforgovernment.org.uk/sites/default/files/timeline-lockdown-web.pdf, all dates supported by Oxford data)

### Milan & Rome
start date: **03/08/2020** (https://milanostyle.com/timeline-of-italy-coronavirus-lockdown/, https://www.axios.com/italy-coronavirus-timeline-lockdown-deaths-cases-2adb0fc7-6ab5-4b7c-9a55-bc6897494dc6.html, very strict, Oxford says 03/10/2020), (**11/04/2020** for red zone & softer lockdown, not fully lifted yet)

end date: **05/04/2020**  (https://easymilano.com/lockdown-phase-2-4th-may-2020-list-of-shops-and-services-opening/, also supported by Oxford data)

### Hong Kong
**No lockdown** only recommend not leaving home, Oxford data

### Seoul
**No lockdown** but according to Oxford data - equire not leaving house with exceptions for daily exercise, grocery shopping, and 'essential' trips from 03/21/2020 to 04/17/2020

### Sao Paulo
**No lockdown** according to Oxford data

### Sydney
No strict lockdown for Sydney, but **12/19/2020** to **01/10/2021** for a part of Syndey (According to Oxford data)

### Johannesburg
start date: **03/27/2020** 

end date: **06/01/2020**

(according to Oxford data, supported by https://en.wikipedia.org/wiki/COVID-19_pandemic_in_South_Africa#Levels)

### Wuhan
start date: **01/23/2020**

end date: **04/08/2020**

(according to https://www.bbc.com/zhongwen/simp/chinese-news-52197004)

### Shanghai
start date: **01/24/2020**

end date: **03/22/2020**

(according to https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Shanghai#Lockdown_and_city_restrictions)

### Beijing
**No strict citywide lockdown** but had many "lcoal closed management" senarios once a case or a suspected case announced. 
(according to https://www.theguardian.com/world/2020/jun/15/beijing-lockdown-tightens-as-new-coronavirus-outbreak-spreads)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8f485a30-0ad3-426a-aa3b-c5e3cbe23db6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>